In [3]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import os
import cv2

In [5]:
print(f"Current Path: {os.getcwd()}")

Current Path: /home/fuyukawa/IAP/Final_Project/GRIK/build/240607/Utils


In [6]:
engine_path = "../Weights/Ver01/best_tiny_400_16.trt"

In [10]:
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
runtime = trt.Runtime(TRT_LOGGER)
trt.init_libnvinfer_plugins(None, "")

True

In [11]:
with open(engine_path, 'rb') as f:
    engine = runtime.deserialize_cuda_engine(f.read())

In [ ]:
context = engine.create_execution_context()

In [ ]:
inputs, outputs, bindings = [], [], []
stream = cuda.Stream()
for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        bindings.append(int(device_mem))
        if engine.binding_is_input(binding):
            inputs.append({'host': host_mem, 'device': device_mem})
        else:
            outputs.append({'host': host_mem, 'device': device_mem})

In [ ]:
# Load image
img = cv2.imread("../test05.jpg")


In [ ]:
[cuda.memcpy_htod_async(inp['device'], inp['host'], stream) for inp in inputs]
context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
[cuda.memcpy_dtoh_async(out['host'], out['device'], stream) for out in outputs]
stream.synchronize()
output = [out['host'] for out in outputs]
print("Inference output: ", output)